# Heisenberg spin chain (with PBC)

$$ H = -\frac{1}{4}\sum\limits_{j=1}^N \left( \frac{J_j^x}{2} \sigma_{j+1}^x\sigma_j^x + \frac{J_j^y}{2} \sigma_{j+1}^y\sigma_j^y + \frac{J_j^z}{2} \sigma_{j+1}^z\sigma_j^z + h_j^x \sigma_j^x + h_j^y \sigma_j^y + h_j^z \sigma_j^z \right) $$ 

In [ ]:
# import libraries

import matplotlib.pyplot as plt
import numpy as np
from qutip import *
import seaborn as sns; sns.set_theme()

plt.rcParams.update({
"text.usetex": True,
"font.family": "sans-serif",
"font.sans-serif": ["Helvetica"], "font.size": 24})